## Future Enhancements / To-do list

* Fix DURATION variable from static to dynamic based on specific file length
* Increase num_segments variable from 5 to 10+ in order to get samples at about 3 seconds each (needs dynamic duration to calculate)

In [27]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [26]:
import os
import librosa
import math
import json

DATASET_PATH = "organizedWavs"
JSON_PATH = "data_static_60_5.json"
SAMPLE_RATE = 22050

## NEED TO UPDATE TO BE DYNAMIC
DURATION = 30 # measured in seconds, but wont work for us in the long term
##

SAMPLES_PER_TRACK = SAMPLE_RATE * DURATION

# identify the specific track duration
## frames, sample rate --> duration

# split the duration into 3 second segments
## send the number of segments into the save_mfcc function
## update the SAMPLES_PER_TRACK variable used in save_mfcc

def save_mfcc(dataset_path, json_path, n_mfcc=13, n_fft=2048, hop_length=512, num_segments=10):
    # build a dictionary to store data
    data = {
        "mapping": [], #"mapping": ["clacky","thocky"], # 0 = clacky, 1 = thocky
        "labels": [], # target outputs
        "mfcc": [] # training inputs
    }
    
    num_samples_per_segment = int(SAMPLES_PER_TRACK / num_segments)
    expected_num_mfcc_vectors_per_segment = math.ceil(num_samples_per_segment / hop_length) # 1.2 -> 2
    
    # loop through all genres
    for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
        print('\n dirpath: {},\n dirnames: {},\n filenames: {}'.format(dirpath,dirnames,filenames))
        
        # ensure that we are not at the root level
        if dirpath is not dataset_path:
            # save the semantic
            dirpath_components = dirpath.split("/") # genre/blues ==> ["genre", "blues"]
            semantic_label = dirpath_components[-1]
            data["mapping"].append(semantic_label)
            print("\n Processing {}".format(semantic_label))
            
            # process files for a specific genre
            for f in filenames:
                
                # load audio file
                file_path = os.path.join(dirpath, f)
                signal, sr = librosa.load(file_path, sr=SAMPLE_RATE)
                f_duration = librosa.get_duration(y=signal, sr=sr)
                
                # num segments is dependent on the duration / 3 second intervals
                new_num_segments = int(f_duration / 3)
                
                # process segments extracting mfcc and storing data
                
                #for s in range(new_num_segments): -- commenting out the dynamic segment piece
                for s in range(num_segments):    
                    # re-calculate based on specific file --commented out
                    #num_samples_per_segment = int((SAMPLE_RATE * f_duration) / new_num_segments)
                    
                    # reverting to static for now
                    num_samples_per_segment = int((SAMPLE_RATE * DURATION) / num_segments)
                    expected_num_mfcc_vectors_per_segment = math.ceil(num_samples_per_segment / hop_length) # 1.2 -> 2
                    #
                    
                    start_sample = num_samples_per_segment * s 
                    finish_sample = start_sample + num_samples_per_segment
                    
                    mfcc = librosa.feature.mfcc(y=signal[start_sample:finish_sample],
                                                sr=sr,
                                                n_fft=n_fft,
                                                n_mfcc=n_mfcc,
                                                hop_length=hop_length
                                               )
                    
                    mfcc = mfcc.T 
                    # store mfcc for segment only if it has the expected length
                    if len(mfcc) == expected_num_mfcc_vectors_per_segment:
                        data["mfcc"].append(mfcc.tolist())
                        data["labels"].append(i-1)
                        print("{}, segment:{}".format(file_path, s))                  
               
    with open(json_path, "w") as fp:
        json.dump(data, fp, indent=4)

if __name__ == "__main__":
    save_mfcc(DATASET_PATH, JSON_PATH, num_segments=60) # num_segments back to being used


 dirpath: organizedWavs,
 dirnames: ['Clacky', 'Clicky', 'Muted', 'Silent', 'Thocky'],
 filenames: []

 dirpath: organizedWavs\Clacky,
 dirnames: [],
 filenames: ['Axiom Studios Alais R2 with Alpaca V2 Switches TypingSound Test.wav', 'Bakeneko - NK Creams - Sound Test.wav', 'Bakeneko60 with Alpacas Typing Sounds.wav', 'BBOX60   Salmon Alps.wav', 'Build #024 - Meridian Typing Test.wav', 'Cannonkeys Lilac Brutal60 with Kiwis Typing Sounds.wav', 'Clacc Clacc!   Bakeneko W Lavenders.wav', 'Gentleman 65 Suited Edition Sound Test (PC And ALU).wav', 'Ikki68 Aurora with Auralite Switches (lubed and filmed) Sound Test.wav', 'Invictus (Private GB version RE.60) with Hyperglides.wav', 'Iron 165 R2 with Cherry Hyperglide Mx Brown.wav', 'Iron180  Smith + Rune  Forged Carbon Fiber Plate  Black Cherry Pie.wav', 'KBD67 Lite Mint R2 - Sound Test v24.0 (Brass Plate  Boba Ice Thoccs  No Mute Silicone on Plate).wav', 'light tactiles are endgame  Bakeneko60 with Durock Light Tactiles Typing Sounds.wav', '

organizedWavs\Clacky\Bakeneko - NK Creams - Sound Test.wav, segment:40
organizedWavs\Clacky\Bakeneko - NK Creams - Sound Test.wav, segment:41
organizedWavs\Clacky\Bakeneko - NK Creams - Sound Test.wav, segment:42
organizedWavs\Clacky\Bakeneko - NK Creams - Sound Test.wav, segment:43
organizedWavs\Clacky\Bakeneko - NK Creams - Sound Test.wav, segment:44
organizedWavs\Clacky\Bakeneko - NK Creams - Sound Test.wav, segment:45
organizedWavs\Clacky\Bakeneko - NK Creams - Sound Test.wav, segment:46
organizedWavs\Clacky\Bakeneko - NK Creams - Sound Test.wav, segment:47
organizedWavs\Clacky\Bakeneko - NK Creams - Sound Test.wav, segment:48
organizedWavs\Clacky\Bakeneko - NK Creams - Sound Test.wav, segment:49
organizedWavs\Clacky\Bakeneko - NK Creams - Sound Test.wav, segment:50
organizedWavs\Clacky\Bakeneko - NK Creams - Sound Test.wav, segment:51
organizedWavs\Clacky\Bakeneko - NK Creams - Sound Test.wav, segment:52
organizedWavs\Clacky\Bakeneko - NK Creams - Sound Test.wav, segment:53
organi

organizedWavs\Clacky\Build #024 - Meridian Typing Test.wav, segment:0
organizedWavs\Clacky\Build #024 - Meridian Typing Test.wav, segment:1
organizedWavs\Clacky\Build #024 - Meridian Typing Test.wav, segment:2
organizedWavs\Clacky\Build #024 - Meridian Typing Test.wav, segment:3
organizedWavs\Clacky\Build #024 - Meridian Typing Test.wav, segment:4
organizedWavs\Clacky\Build #024 - Meridian Typing Test.wav, segment:5
organizedWavs\Clacky\Build #024 - Meridian Typing Test.wav, segment:6
organizedWavs\Clacky\Build #024 - Meridian Typing Test.wav, segment:7
organizedWavs\Clacky\Build #024 - Meridian Typing Test.wav, segment:8
organizedWavs\Clacky\Build #024 - Meridian Typing Test.wav, segment:9
organizedWavs\Clacky\Build #024 - Meridian Typing Test.wav, segment:10
organizedWavs\Clacky\Build #024 - Meridian Typing Test.wav, segment:11
organizedWavs\Clacky\Build #024 - Meridian Typing Test.wav, segment:12
organizedWavs\Clacky\Build #024 - Meridian Typing Test.wav, segment:13
organizedWavs\Cl

organizedWavs\Clacky\Clacc Clacc!   Bakeneko W Lavenders.wav, segment:0
organizedWavs\Clacky\Clacc Clacc!   Bakeneko W Lavenders.wav, segment:1
organizedWavs\Clacky\Clacc Clacc!   Bakeneko W Lavenders.wav, segment:2
organizedWavs\Clacky\Clacc Clacc!   Bakeneko W Lavenders.wav, segment:3
organizedWavs\Clacky\Clacc Clacc!   Bakeneko W Lavenders.wav, segment:4
organizedWavs\Clacky\Clacc Clacc!   Bakeneko W Lavenders.wav, segment:5
organizedWavs\Clacky\Clacc Clacc!   Bakeneko W Lavenders.wav, segment:6
organizedWavs\Clacky\Clacc Clacc!   Bakeneko W Lavenders.wav, segment:7
organizedWavs\Clacky\Clacc Clacc!   Bakeneko W Lavenders.wav, segment:8
organizedWavs\Clacky\Clacc Clacc!   Bakeneko W Lavenders.wav, segment:9
organizedWavs\Clacky\Clacc Clacc!   Bakeneko W Lavenders.wav, segment:10
organizedWavs\Clacky\Clacc Clacc!   Bakeneko W Lavenders.wav, segment:11
organizedWavs\Clacky\Clacc Clacc!   Bakeneko W Lavenders.wav, segment:12
organizedWavs\Clacky\Clacc Clacc!   Bakeneko W Lavenders.wav,

organizedWavs\Clacky\Ikki68 Aurora with Auralite Switches (lubed and filmed) Sound Test.wav, segment:0
organizedWavs\Clacky\Ikki68 Aurora with Auralite Switches (lubed and filmed) Sound Test.wav, segment:1
organizedWavs\Clacky\Ikki68 Aurora with Auralite Switches (lubed and filmed) Sound Test.wav, segment:2
organizedWavs\Clacky\Ikki68 Aurora with Auralite Switches (lubed and filmed) Sound Test.wav, segment:3
organizedWavs\Clacky\Ikki68 Aurora with Auralite Switches (lubed and filmed) Sound Test.wav, segment:4
organizedWavs\Clacky\Ikki68 Aurora with Auralite Switches (lubed and filmed) Sound Test.wav, segment:5
organizedWavs\Clacky\Ikki68 Aurora with Auralite Switches (lubed and filmed) Sound Test.wav, segment:6
organizedWavs\Clacky\Ikki68 Aurora with Auralite Switches (lubed and filmed) Sound Test.wav, segment:7
organizedWavs\Clacky\Ikki68 Aurora with Auralite Switches (lubed and filmed) Sound Test.wav, segment:8
organizedWavs\Clacky\Ikki68 Aurora with Auralite Switches (lubed and film

...

organizedWavs\Thocky\Bakeneko with Gateron Hippos Typing Test.wav, segment:42
organizedWavs\Thocky\Bakeneko with Gateron Hippos Typing Test.wav, segment:43
organizedWavs\Thocky\Bakeneko with Gateron Hippos Typing Test.wav, segment:44
organizedWavs\Thocky\Bakeneko with Gateron Hippos Typing Test.wav, segment:45
organizedWavs\Thocky\Bakeneko with Gateron Hippos Typing Test.wav, segment:46
organizedWavs\Thocky\Bakeneko with Gateron Hippos Typing Test.wav, segment:47
organizedWavs\Thocky\Bakeneko with Gateron Hippos Typing Test.wav, segment:48
organizedWavs\Thocky\Bakeneko with Gateron Hippos Typing Test.wav, segment:49
organizedWavs\Thocky\Bakeneko with Gateron Hippos Typing Test.wav, segment:50
organizedWavs\Thocky\Bakeneko with Gateron Hippos Typing Test.wav, segment:51
organizedWavs\Thocky\Bakeneko with Gateron Hippos Typing Test.wav, segment:52
organizedWavs\Thocky\Bakeneko with Gateron Hippos Typing Test.wav, segment:53
organizedWavs\Thocky\Bakeneko with Gateron Hippos Typing Test.wa

organizedWavs\Thocky\Gentleman65 SE with Momoka Frogs Typing Sound Test (FR4 Plate).wav, segment:0
organizedWavs\Thocky\Gentleman65 SE with Momoka Frogs Typing Sound Test (FR4 Plate).wav, segment:1
organizedWavs\Thocky\Gentleman65 SE with Momoka Frogs Typing Sound Test (FR4 Plate).wav, segment:2
organizedWavs\Thocky\Gentleman65 SE with Momoka Frogs Typing Sound Test (FR4 Plate).wav, segment:3
organizedWavs\Thocky\Gentleman65 SE with Momoka Frogs Typing Sound Test (FR4 Plate).wav, segment:4
organizedWavs\Thocky\Gentleman65 SE with Momoka Frogs Typing Sound Test (FR4 Plate).wav, segment:5
organizedWavs\Thocky\Gentleman65 SE with Momoka Frogs Typing Sound Test (FR4 Plate).wav, segment:6
organizedWavs\Thocky\Gentleman65 SE with Momoka Frogs Typing Sound Test (FR4 Plate).wav, segment:7
organizedWavs\Thocky\Gentleman65 SE with Momoka Frogs Typing Sound Test (FR4 Plate).wav, segment:8
organizedWavs\Thocky\Gentleman65 SE with Momoka Frogs Typing Sound Test (FR4 Plate).wav, segment:9
organizedW

organizedWavs\Thocky\Soundtest  Ep. 26  JWICK ALOE  IKKI AURORA 68.wav, segment:38
organizedWavs\Thocky\Soundtest  Ep. 26  JWICK ALOE  IKKI AURORA 68.wav, segment:39
organizedWavs\Thocky\Soundtest  Ep. 26  JWICK ALOE  IKKI AURORA 68.wav, segment:40
organizedWavs\Thocky\Soundtest  Ep. 26  JWICK ALOE  IKKI AURORA 68.wav, segment:41
organizedWavs\Thocky\Soundtest  Ep. 26  JWICK ALOE  IKKI AURORA 68.wav, segment:42
organizedWavs\Thocky\Soundtest  Ep. 26  JWICK ALOE  IKKI AURORA 68.wav, segment:43
organizedWavs\Thocky\Soundtest  Ep. 26  JWICK ALOE  IKKI AURORA 68.wav, segment:44
organizedWavs\Thocky\Soundtest  Ep. 26  JWICK ALOE  IKKI AURORA 68.wav, segment:45
organizedWavs\Thocky\Soundtest  Ep. 26  JWICK ALOE  IKKI AURORA 68.wav, segment:46
organizedWavs\Thocky\Soundtest  Ep. 26  JWICK ALOE  IKKI AURORA 68.wav, segment:47
organizedWavs\Thocky\Soundtest  Ep. 26  JWICK ALOE  IKKI AURORA 68.wav, segment:48
organizedWavs\Thocky\Soundtest  Ep. 26  JWICK ALOE  IKKI AURORA 68.wav, segment:49
orga

organizedWavs\Thocky\ThockyCreamy Keyboard ($50 Goodness).wav, segment:0
organizedWavs\Thocky\ThockyCreamy Keyboard ($50 Goodness).wav, segment:1
organizedWavs\Thocky\ThockyCreamy Keyboard ($50 Goodness).wav, segment:2
organizedWavs\Thocky\ThockyCreamy Keyboard ($50 Goodness).wav, segment:3
organizedWavs\Thocky\ThockyCreamy Keyboard ($50 Goodness).wav, segment:4
organizedWavs\Thocky\ThockyCreamy Keyboard ($50 Goodness).wav, segment:5
organizedWavs\Thocky\ThockyCreamy Keyboard ($50 Goodness).wav, segment:6
organizedWavs\Thocky\ThockyCreamy Keyboard ($50 Goodness).wav, segment:7
organizedWavs\Thocky\ThockyCreamy Keyboard ($50 Goodness).wav, segment:8
organizedWavs\Thocky\ThockyCreamy Keyboard ($50 Goodness).wav, segment:9
organizedWavs\Thocky\ThockyCreamy Keyboard ($50 Goodness).wav, segment:10
organizedWavs\Thocky\ThockyCreamy Keyboard ($50 Goodness).wav, segment:11
organizedWavs\Thocky\ThockyCreamy Keyboard ($50 Goodness).wav, segment:12
organizedWavs\Thocky\ThockyCreamy Keyboard ($50 

organizedWavs\Thocky\[Rebuild] Build #023 - Sangeo60 Azure Dragon Typing Test.wav, segment:0
organizedWavs\Thocky\[Rebuild] Build #023 - Sangeo60 Azure Dragon Typing Test.wav, segment:1
organizedWavs\Thocky\[Rebuild] Build #023 - Sangeo60 Azure Dragon Typing Test.wav, segment:2
organizedWavs\Thocky\[Rebuild] Build #023 - Sangeo60 Azure Dragon Typing Test.wav, segment:3
organizedWavs\Thocky\[Rebuild] Build #023 - Sangeo60 Azure Dragon Typing Test.wav, segment:4
organizedWavs\Thocky\[Rebuild] Build #023 - Sangeo60 Azure Dragon Typing Test.wav, segment:5
organizedWavs\Thocky\[Rebuild] Build #023 - Sangeo60 Azure Dragon Typing Test.wav, segment:6
organizedWavs\Thocky\[Rebuild] Build #023 - Sangeo60 Azure Dragon Typing Test.wav, segment:7
organizedWavs\Thocky\[Rebuild] Build #023 - Sangeo60 Azure Dragon Typing Test.wav, segment:8
organizedWavs\Thocky\[Rebuild] Build #023 - Sangeo60 Azure Dragon Typing Test.wav, segment:9
organizedWavs\Thocky\[Rebuild] Build #023 - Sangeo60 Azure Dragon Typi